In [0]:
client_id = dbutils.secrets.get(scope="formula1keyvolt1 ", key="client-id")
tenant_id = dbutils.secrets.get(scope="formula1keyvolt1 ", key="tenant-id")
client_secret = dbutils.secrets.get(scope="formula1keyvolt1 ", key="client-secret")

In [0]:
container_name = 'demo'
account_name = 'formula1projectdl2'
mount_point = '/mnt/demo'

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = f"abfss://{container_name}@{account_name}.dfs.core.windows.net/",
  mount_point = mount_point,
  extra_configs = configs)

True

In [0]:
circuits_path = "/mnt/demo/circuits.csv"
cir_df=spark.read.csv(path=circuits_path, header=True)

In [0]:
display(cir_df)

In [0]:
def mount_adls(storage_acc_name, container_name):
    #Get secrets from Kei Vault
    client_id = dbutils.secrets.get(scope="formula1keyvolt1 ", key="client-id")
    tenant_id = dbutils.secrets.get(scope="formula1keyvolt1 ", key="tenant-id")
    client_secret = dbutils.secrets.get(scope="formula1keyvolt1 ", key="client-secret")
    
    #Set spark configuration
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    # Unmount if container is already mounted so it does not show error
    if any(mount.mountPoint == f"/mnt/{storage_acc_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_acc_name}/{container_name}")

    # Mount the storage account container
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_acc_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_acc_name}/{container_name}",
        extra_configs = configs)
    
    display(dbutils.fs.mounts())

### Mount Raw container

In [0]:
mount_adls("formula1projectdl2","raw")

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/formula1projectdl2/raw,abfss://raw@formula1projectdl2.dfs.core.windows.net/,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/mnt/demo,abfss://demo@formula1projectdl2.dfs.core.windows.net/,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,


### mount Processed container

In [0]:
mount_adls("formula1projectdl2","processed")

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/formula1projectdl2/raw,abfss://raw@formula1projectdl2.dfs.core.windows.net/,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/mnt/demo,abfss://demo@formula1projectdl2.dfs.core.windows.net/,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,


In [0]:
mount_adls("formula1projectdl2","presentation")

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/formula1projectdl2/raw,abfss://raw@formula1projectdl2.dfs.core.windows.net/,
/mnt/formula1projectdl2/presentation,abfss://presentation@formula1projectdl2.dfs.core.windows.net/,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/mnt/demo,abfss://demo@formula1projectdl2.dfs.core.windows.net/,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
